# Generating the data
<!-- 1. Generate the Heterogeneous graph -->
<!-- 2. Generate the feature set from the clinical notes. -->
<!-- 3. Generate the Labels -->
1. read the HG
2. Generate the k-metapath-based similarity matrices
3. Convert the As to edge-based.
<!-- 6. Downsize the X to a vector of 32 values. -->

In [1]:
import sys, os, copy
import random
from copy import deepcopy
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import scipy.sparse as sp


sys.path.append('../data_generation')

from  ModelFunctions import *
# ============================================================================
DISEASE_FILE = 'all_MIMIC'
print(f"Disease File: {DISEASE_FILE}")
# ============================================================================


## Athena

# OAK
HG_path = f'../Data/HG'

# reading the Nodes

with open(f'{HG_path}/Nodes.pkl', 'rb') as file:
    Nodes = pickle.load(file)

X = torch.load(f'{HG_path}/X.pt')
Y = torch.load(f'{HG_path}/Y.pt')

HG = nx.read_gml(f'{HG_path}/HG.gml')

Disease File: all_MIMIC


/tmp/ipykernel_770446/1435998400.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X = torch.load(f'{HG_path}/X.pt')
/tmp/ipykernel_770446/1435998400.py:30: FutureWarning

In [2]:
import os
import pickle

def save_list_as_pickle(L, given_path, file_name):
    # Check if the directory exists, if not, create it
    if not os.path.exists(given_path):
        os.makedirs(given_path)
        print(f'Directory {given_path} created.')
    
    # Save the list as a pickle file
    print(f'Saving to {given_path}/{file_name}.pkl')
    with open(f'{given_path}/{file_name}.pkl', 'wb') as file:
        pickle.dump(L, file)

In [3]:
def G_statistics(G):
    Nodes = list(G.nodes())

    Patients =    [v for v in Nodes if v[0]=='C']
    Visits =      [v for v in Nodes if v[0]=='V']
    Medications = [v for v in Nodes if v[0]=='M']
    Diagnosis =   [v for v in Nodes if v[0]=='D']
    Procedures =  [v for v in Nodes if v[0]=='P']

    print(f'number of patients = {len(Patients)}')
    print(f'number of visits = {len(Visits)}')
    print(f'number of Medication = {len(Medications)}')
    print(f'number of Diagnoses = {len(Diagnosis)}')
    print(f'number of Procedures = {len(Procedures)}')
    print(f'number of Edges = {G.number_of_edges()}')
    
    print('------------------------------------------\n')

def calculate_percentage_of_zeros(A):
    """
    Calculate the percentage of zero values in a 2D numpy array.

    Parameters:
    A (numpy.ndarray): 2D numpy array

    Returns:
    float: Percentage of zero values
    """
    # Count the number of zero values
    num_zeros = np.count_nonzero(A == 0)

    # Count the total number of values
    total_values = A.size

    # Calculate the percentage of zero values
    percentage_zeros = (num_zeros / total_values) * 100

    return percentage_zeros

# Removing <patients to delete> from HG
def remove_patients_and_linked_visits(nodes, HG):
    '''remove patients and their visits from HG'''
    print('Number of PATIENTS to remove: ', len(nodes))
    
    new_HG = deepcopy(HG)
    nodes_to_remove = []
    for node in nodes:
        for v in HG.neighbors(node):
            if v[0]=='V':
                nodes_to_remove.append(v)
        nodes_to_remove.append(node)
    print('Number of nodes to remove: ', len(nodes_to_remove))
    new_HG.remove_nodes_from(nodes_to_remove)
    return new_HG    

## Sample of patients....


In [4]:
disease_name = 'infectious'
with open(f'{HG_path}/Patients_{disease_name}.pkl', 'rb') as file:
    Sample_Patients = pickle.load(file)

# ==============================================================
# Updating the HG by removing other patients
Patients, Visits, Medications, Diagnosis, Procedures, _ = get_Nodes(HG)
Nodes = Patients + Visits + Medications + Diagnosis  + Procedures

old_indeces = {p: i for i, p in enumerate(Nodes)}

# Removing patients and linked information...
nodes_to_delete = [n for n in Patients if n not in Sample_Patients]
G_statistics(HG)    
new_HG = remove_patients_and_linked_visits(nodes_to_delete, HG)
G_statistics(new_HG)    
del HG
HG = deepcopy(new_HG)
del new_HG

# =======================================================================
Patients, Visits, Medications, Diagnosis, Procedures, _ = get_Nodes(HG)
Nodes = Patients + Visits + Medications + Diagnosis  + Procedures

new_indeces = [old_indeces[v] for v in Nodes]

NewX = X[new_indeces]
NewY = Y[new_indeces]

# ============================================================================
saving_path = f'../Data/203_Diagnoses/{disease_name}'

torch.save(NewX, f'{saving_path}/X.pt')
torch.save(NewY, f'{saving_path}/Y.pt')



number of patients = 46437
number of visits = 58897
number of Medication = 592
number of Diagnoses = 203
number of Procedures = 89
number of patients = 46437
number of visits = 58897
number of Medication = 592
number of Diagnoses = 203
number of Procedures = 89
number of Edges = 769929
------------------------------------------

Number of PATIENTS to remove:  34862
Number of nodes to remove:  74349
number of patients = 11575
number of visits = 19410
number of Medication = 592
number of Diagnoses = 203
number of Procedures = 89
number of Edges = 325301
------------------------------------------

number of patients = 11575
number of visits = 19410
number of Medication = 592
number of Diagnoses = 203
number of Procedures = 89


## Visits Splitting...

In [5]:
Patients, Visits, Medications, Diagnosis, Procedures, _ = get_Nodes(HG)
Nodes2 = Patients + Visits + Medications + Diagnosis  + Procedures
# # =============================================================================
# # =============================================================================

np.random.seed(42)
P_train, P_temp = train_test_split(Visits, test_size=0.8, random_state=42)
P_val, P_test = train_test_split(P_temp, test_size=1/2, random_state=42)

len(P_train)/len(Patients), len(P_test)/len(Patients), len(P_val)/len(Patients)

# ============================================================================
save_list_as_pickle(P_train, saving_path, 'V_train')
save_list_as_pickle(P_test,  saving_path, 'V_test')
save_list_as_pickle(P_val,   saving_path, 'V_val')
save_list_as_pickle(Nodes2,  saving_path, 'Nodes')



number of patients = 11575
number of visits = 19410
number of Medication = 592
number of Diagnoses = 203
number of Procedures = 89
Saving to ../Data/203_Diagnoses/infectious/V_train.pkl
Saving to ../Data/203_Diagnoses/infectious/V_test.pkl
Saving to ../Data/203_Diagnoses/infectious/V_val.pkl
Saving to ../Data/203_Diagnoses/infectious/Nodes.pkl


In [6]:
def subset_adjacency_matrix(G, all_nodes, subset_nodes):
    """
    Returns the adjacency matrix of a subset of nodes in the graph G,
    keeping all other nodes in the matrix but without edges.
    
    Parameters:
    G (networkx.Graph): The input graph.
    subset_nodes (list): The list of nodes to keep edges for.
    
    Returns:
    numpy.ndarray: The modified adjacency matrix.
    """
    # Get the full adjacency matrix
    adj_matrix = nx.to_numpy_array(G)
    
    # Create a boolean mask to identify nodes not in the subset
    mask = np.isin(all_nodes, subset_nodes)
    
    # For nodes not in the subset, set rows and columns in the adjacency matrix to zero
    for i in range(len(all_nodes)):
        if not mask[i]:
            adj_matrix[i, :] = 0  # Zero out the row
            adj_matrix[:, i] = 0  # Zero out the column
    
    return adj_matrix

In [7]:
# P_train, Visits, Medications, Diagnosis, Procedures, _ = get_Nodes(HG)
# Nodes = P_train + Visits + Medications + Diagnosis  + Procedures

# W_cv = subset_adjacency_matrix(HG, Nodes, P_train + Visits)
W_cv = subset_adjacency_matrix(HG, Nodes2, Patients + Visits)
W_vm = subset_adjacency_matrix(HG, Nodes2, Visits + Medications)
W_vd = subset_adjacency_matrix(HG, Nodes2, Visits + Diagnosis)
W_vp = subset_adjacency_matrix(HG, Nodes2, Visits + Procedures)  


print([a.shape for a in [W_cv, W_vm, W_vd, W_vp]])

[(31869, 31869), (31869, 31869), (31869, 31869), (31869, 31869)]


In [8]:
from scipy.sparse import csr_matrix, vstack
from joblib import Parallel, delayed

def parallel_multiply_chunk(W1_csr, W2_csr, row_indices):
    # Multiply a chunk of rows from W1_csr with W2_csr
    result_chunk = W1_csr[row_indices].dot(W2_csr)
    return result_chunk

def M(W1, W2, n_jobs=-1):
    # Convert to CSR format if not already
    W1_csr = csr_matrix(W1) if not isinstance(W1, csr_matrix) else W1
    W2_csr = csr_matrix(W2) if not isinstance(W2, csr_matrix) else W2

    # Get the number of rows in W1
    n_rows = W1_csr.shape[0]

    # Determine the chunk size per job
    chunk_size = n_rows // n_jobs if n_jobs > 1 else n_rows

    # Create row indices to split the workload
    row_chunks = [range(i, min(i + chunk_size, n_rows)) for i in range(0, n_rows, chunk_size)]

    # Use Parallel to distribute the computation
    print(f'multiplying {W1.shape} * {W2.shape} in parallel...')
    results = Parallel(n_jobs=n_jobs)(
        delayed(parallel_multiply_chunk)(W1_csr, W2_csr, row_indices) for row_indices in row_chunks
    )

    # Stack the results vertically as a sparse matrix
    result = vstack(results)
    print("Done multiplication...")

    return result

# Example usage:
# Assuming W1 and W2 are large sparse matrices
# W1 = ...
# W2 = ...
# Result = M(W1, W2, n_jobs=4)  # Example with 4 parallel jobs


## PathCount

PathCount here is as follow:
for meta-path $p = CVMVC$, we say: $$PC_p = A_{CV} \times A_{VM} \times A_{VM}^T \times A_{CV}^T $$
This is equivalent to 
$$A_{CVM} = A_{CV} \times A_{VM}$$
$$PC_p = A_{CVM} \times A_{CVM}^T $$

In [9]:
# PathCount 
# Heterogeneous similarity

W_CVM = M(W_cv, W_vm)
W_CVD = M(W_cv, W_vd)
W_CVP = M(W_cv, W_vp)

W_DVM = M(W_vd.T, W_vm)
W_DVP = M(W_vd.T, W_vp)
W_PVM = M(W_vp.T, W_vm)

# Homogeneous similarity
# W_CVMVC = M(M(M(W_cv, W_vm), W_vm.T), W_cv.T)
# W_CVDVC = M(M(M(W_cv, W_vd), W_vd.T), W_cv.T)
# W_CVPVC = M(M(M(W_cv, W_vp), W_vp.T), W_cv.T)
# W_DVMVD = M(M(M(W_vd.T, W_vm), W_vm.T), W_vd)
# W_DVPVD = M(M(M(W_vd.T, W_vp), W_vp.T), W_vd)
# W_MVDVM = M(M(M(W_vm.T, W_vd), W_vd.T), W_vm)
# W_MVPVM = M(M(M(W_vm.T, W_vp), W_vp.T), W_vm)
# W_PVDVP = M(M(M(W_vp.T, W_vd), W_vd.T), W_vp)
# W_PVMVP = M(M(M(W_vp.T, W_vm), W_vm.T), W_vp)

W_CVMVC = M(W_CVM, W_CVM.T)
W_CVDVC = M(W_CVD, W_CVD.T)
W_CVPVC = M(W_CVP, W_CVP.T)
print('Patient-Patient completed!\n')

W_VMV = M(W_vm, W_vm.T)
W_VDV = M(W_vd, W_vd.T)
W_VPV = M(W_vp, W_vp.T)
print('visit-visit completed!\n')

W_DVMVD = M(W_DVM, W_DVM.T)
W_DVPVD = M(W_DVP, W_DVP.T)
print('Diagnoses-Diagnoses completed!\n')

W_MVDVM = M(W_DVM.T, W_DVM)
W_MVPVM = M(W_PVM.T, W_PVM)
print('Med-Med completed!\n')

W_PVDVP = M(W_DVP.T, W_DVP)
W_PVMVP = M(W_PVM, W_PVM.T)
print('Proced-Proced completed!\n')

multiplying (31869, 31869) * (31869, 31869) in parallel...
Done multiplication...
multiplying (31869, 31869) * (31869, 31869) in parallel...
Done multiplication...
multiplying (31869, 31869) * (31869, 31869) in parallel...
Done multiplication...
multiplying (31869, 31869) * (31869, 31869) in parallel...
Done multiplication...
multiplying (31869, 31869) * (31869, 31869) in parallel...
Done multiplication...
multiplying (31869, 31869) * (31869, 31869) in parallel...
Done multiplication...
multiplying (31869, 31869) * (31869, 31869) in parallel...
Done multiplication...
multiplying (31869, 31869) * (31869, 31869) in parallel...
Done multiplication...
multiplying (31869, 31869) * (31869, 31869) in parallel...
Done multiplication...
Patient-Patient completed!

multiplying (31869, 31869) * (31869, 31869) in parallel...
Done multiplication...
multiplying (31869, 31869) * (31869, 31869) in parallel...
Done multiplication...
multiplying (31869, 31869) * (31869, 31869) in parallel...
Done multip

## Symmetric PathSim (SPS)

In [10]:
def symmetricPathSim(G, p, Nodes):
    '''SPS
        G: heterogeneous graph,
       p: meta-path, 
       |p| = 3,
       return A(N by N).'''
    n = len(Nodes)
    A = np.zeros((n, n))
    for i in range(0, n-1):
        ni = Nodes[i]
        if ni[0] == p[0]:
            Ni = [x for x in G.neighbors(ni) if x[0]== p[1]]
            for j in range(i+1, n):
                nj = Nodes[j]
                if nj[0] == p[-1]:
                    Nj = [x for x in G.neighbors(nj) if x[0]== p[1]]
                    # Now, we count len of common neighbors
                    top_value = (len(set(Ni) & set(Nj)) + len(set(Nj) & set(Ni)))
                    button_value = (len(set(Ni) & set(Ni)) + len(set(Nj) & set(Nj)))
                    if button_value > 0 :
                        A[i,j] =  top_value / button_value 
                        A[j,i] =  A[i,j]
                    else:
                        A[i,j] = 0
                        A[j,i] = 0
                    
    return A

def symmetricPathSim_2(PC, Nodes, selected_nodes):
    '''SPS
        G: heterogeneous graph,
       p: meta-path, 
       |p| = 3,
       return A(N by N).'''
       
    selected_indeces = [Nodes.index(n) for n in selected_nodes]
    n = len(Nodes)
    SPS = np.zeros((n, n))
    for i in range(0, len(selected_nodes)-1):
        ni = selected_indeces[i]
        for j in range(i+1, len(selected_nodes)):
            nj = selected_indeces[j]
            SPS[ni, nj] = (PC[ni, nj] + PC[nj, ni]) / (PC[ni,ni] + PC[nj,nj])
            SPS[nj, ni] = SPS[ni, nj]  # Ensure symmetry
    return SPS

import numpy as np
from joblib import Parallel, delayed, parallel_backend

def calculate_sps(ni, nj):
    # Check for zero diagonals to prevent division by zero
    if PC_shared[ni, ni] + PC_shared[nj, nj] == 0:
        return 0  # or handle in another way, e.g., return np.nan or a small constant

    return (PC_shared[ni, nj] + PC_shared[nj, ni]) / (PC_shared[ni, ni] + PC_shared[nj, nj])

def symmetricPathSim_3(PC, Nodes, selected_nodes):
    '''SPS
        G: heterogeneous graph,
       p: meta-path, 
       |p| = 3,
       return A(N by N).'''
       
    global PC_shared
    PC_shared = PC  # Use shared memory

    selected_indeces = [Nodes.index(n) for n in selected_nodes]
    n = len(Nodes)
    SPS = np.zeros((n, n))

    # Prepare the pairs of indices for parallel processing
    index_pairs = [(i, j) for i in range(len(selected_indeces) - 1)
                          for j in range(i + 1, len(selected_indeces))]

    # Use a backend that supports shared memory
    with parallel_backend('loky', n_jobs=40):
        results = Parallel()(delayed(calculate_sps)(selected_indeces[i], selected_indeces[j])
                             for i, j in index_pairs)

    # Populate the SPS matrix with the computed results
    for idx, (i, j) in enumerate(index_pairs):
        ni, nj = selected_indeces[i], selected_indeces[j]
        SPS[ni, nj] = results[idx]
        SPS[nj, ni] = results[idx]  # Ensure symmetry

    return SPS



# SPS_CVMVC = symmetricPathSim_3(W_CVMVC, Nodes2, P_train)
# SPS_CVDVC = symmetricPathSim_3(W_CVDVC, Nodes2, P_train)
# SPS_CVPVC = symmetricPathSim_3(W_CVPVC, Nodes2, P_train)
# print('Patients-Patients completed!\n')

# SPS_VMV = symmetricPathSim_3(W_VMV, Nodes2, Visits)
# SPS_VDV = symmetricPathSim_3(W_VDV, Nodes2, Visits)
# SPS_VPV = symmetricPathSim_3(W_VPV, Nodes2, Visits)
# print('visit-visit completed!\n')

# SPS_DVMVD = symmetricPathSim_3(W_DVMVD, Nodes2, Diagnosis)
# SPS_DVPVD = symmetricPathSim_3(W_DVPVD, Nodes2, Diagnosis)
# print('Diagnoses-Diagnoses completed!\n')

# SPS_MVDVM = symmetricPathSim_3(W_MVDVM, Nodes2, Medications)
# SPS_MVPVM = symmetricPathSim_3(W_MVPVM, Nodes2, Medications)
# print('Med-Med completed!\n')

# SPS_PVDVP = symmetricPathSim_3(W_PVDVP, Nodes2, Procedures)
# SPS_PVMVP = symmetricPathSim_3(W_PVMVP, Nodes2, Procedures)
# print('Proced-Proced completed!\n')

In [11]:
def keep_top_million(sparse_matrix, top_n=1000000):
    # Convert to CSR if the matrix is dense
    if isinstance(sparse_matrix, np.ndarray):
        sparse_matrix = sparse.csr_matrix(sparse_matrix)
    
    # Step 1: Find the threshold for the top million values
    if sparse_matrix.nnz <= top_n:
        # If the matrix has fewer non-zeros than top_n, return it as is
        return sparse_matrix
    
    # Extract the non-zero data from the sparse matrix
    sorted_data = np.sort(sparse_matrix.data)[-top_n]
    
    # Step 2: Filter the sparse matrix based on this threshold
    mask = sparse_matrix.data >= sorted_data
    
    # Apply the mask to keep only values in the top million
    filtered_data = sparse_matrix.data[mask]
    filtered_indices = sparse_matrix.indices[mask]
    filtered_indptr = np.zeros(sparse_matrix.shape[0] + 1, dtype=int)
    
    # Recalculate indptr array based on the filtered data
    current_index = 0
    for i in range(sparse_matrix.shape[0]):
        row_start = sparse_matrix.indptr[i]
        row_end = sparse_matrix.indptr[i+1]
        
        # Count non-zero elements in the current row that are in the top million
        filtered_indptr[i+1] = filtered_indptr[i] + np.sum(mask[row_start:row_end])
    
    # Create a new sparse matrix with the filtered data
    filtered_matrix = sparse.csr_matrix((filtered_data, filtered_indices, filtered_indptr), shape=sparse_matrix.shape)
    
    return filtered_matrix



As_path = f'{saving_path}/A'
if not os.path.exists(As_path):
    os.makedirs(As_path)


Ws = [W_CVMVC, W_CVDVC, W_CVPVC,          
      W_VMV, W_VDV, W_VPV,          
      W_DVMVD, W_DVPVD,         
      W_MVPVM, W_MVDVM,        
      W_PVDVP, W_PVMVP,        
      W_cv, W_vd, W_vm, W_vp,       
      W_CVM, W_CVD, W_CVP,      
      W_DVM, W_PVM, W_DVP]

# Ws = [SPS_CVMVC, SPS_CVDVC, SPS_CVPVC,          
#       SPS_VMV, SPS_VDV, SPS_VPV,          
#       SPS_DVMVD, SPS_DVPVD,         
#       SPS_MVPVM, SPS_MVDVM,        
#       SPS_PVDVP, SPS_PVMVP,
#       W_cv, W_vd, W_vm, W_vp,       
#       W_CVM, W_CVD, W_CVP,      
#       W_DVM, W_PVM, W_DVP]
      

print(f'Number of meta-paths = {len(Ws)}')

num_As = 0
from scipy.sparse import issparse
for i, A in enumerate(Ws):
    # Check if A is sparse or dense
    if issparse(A):
        non_zero_count = A.count_nonzero()
    else:
        non_zero_count = np.count_nonzero(A)
    
    print(f"Matrix {i}: {non_zero_count} non-zero elements")
    if non_zero_count > 0:
        if non_zero_count > 1000000:
            B = keep_top_million(A, top_n=1000000)
            print(f"\tSaving one million non-zero values... (after reduction: {B.count_nonzero()} non-zero elements)")
        else:
            B = A
            print(f"\tSaving all non-zero values... ({non_zero_count} non-zero elements)")

        # Convert to CSR format if not already sparse
        if not issparse(B):
            B = sparse.csr_matrix(B)
        
        # Save the sparse matrix
        sparse.save_npz(f"{As_path}/sparse_matrix_{num_As}.npz", B)
        num_As+=1

Number of meta-paths = 22
Matrix 0: 1677 non-zero elements
	Saving all non-zero values... (1677 non-zero elements)
Matrix 1: 116239843 non-zero elements
	Saving one million non-zero values... (after reduction: 1023890 non-zero elements)
Matrix 2: 86025154 non-zero elements
	Saving one million non-zero values... (after reduction: 1078408 non-zero elements)
Matrix 3: 55665 non-zero elements
	Saving all non-zero values... (55665 non-zero elements)
Matrix 4: 291137468 non-zero elements
	Saving one million non-zero values... (after reduction: 1037661 non-zero elements)
Matrix 5: 173743531 non-zero elements
	Saving one million non-zero values... (after reduction: 1783166 non-zero elements)
Matrix 6: 20445 non-zero elements
	Saving all non-zero values... (20445 non-zero elements)
Matrix 7: 41209 non-zero elements
	Saving all non-zero values... (41209 non-zero elements)
Matrix 8: 187264 non-zero elements
	Saving all non-zero values... (187264 non-zero elements)
Matrix 9: 200765 non-zero elemen

## 5. Convert the As to edge-based.

In [12]:
# num_As = len(Ws)
# Check if the directory exists, and create it if it doesn't
Nodes_path = f'{saving_path}/edges'
if not os.path.exists(Nodes_path):
    os.makedirs(Nodes_path)

D = [get_edges_dict(f'{saving_path}/A/sparse_matrix_{i}.npz') for i in range(len(Ws)-1)]
sorted_list_of_dicts = sorted(D, key=lambda x: len(x), reverse=True)

unique_edges = set()

for e in sorted_list_of_dicts:
    unique_edges.update(e.keys())

    
with open(f'{saving_path}/edges/edge_list.pkl', 'wb') as file:
    pickle.dump(unique_edges, file)

print('done saving [unique edges]: ', len(unique_edges))

for i, d in enumerate(D):
    print(f'Working on {i}th file...')
    results = []
    for e in unique_edges:
        if e in d:
            results.append(d[e])
        else:
            results.append(0)
    print(f'\tdone...')
    print('\tSaving...')

    with open(f'{saving_path}/edges/edge_weight{i}.pkl', 'wb') as file:
        pickle.dump(results, file)

    print(f'\tDone saving edge_weight{i}...')
    


done saving [unique edges]:  2909216
Working on 0th file...
	done...
	Saving...
	Done saving edge_weight0...
Working on 1th file...
	done...
	Saving...
	Done saving edge_weight1...
Working on 2th file...
	done...
	Saving...
	Done saving edge_weight2...
Working on 3th file...
	done...
	Saving...
	Done saving edge_weight3...
Working on 4th file...
	done...
	Saving...
	Done saving edge_weight4...
Working on 5th file...
	done...
	Saving...
	Done saving edge_weight5...
Working on 6th file...
	done...
	Saving...
	Done saving edge_weight6...
Working on 7th file...
	done...
	Saving...
	Done saving edge_weight7...
Working on 8th file...
	done...
	Saving...
	Done saving edge_weight8...
Working on 9th file...
	done...
	Saving...
	Done saving edge_weight9...
Working on 10th file...
	done...
	Saving...
	Done saving edge_weight10...
Working on 11th file...
	done...
	Saving...
	Done saving edge_weight11...
Working on 12th file...
	done...
	Saving...
	Done saving edge_weight12...
Working on 13th file.